# siungle variable 

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load your data
# Assuming data is a pandas DataFrame with a single column
data = pd.read_csv('your_time_series_data.csv')
series = data['your_column_name'].values

# Feature Scaling
scaler = MinMaxScaler()
series_scaled = scaler.fit_transform(series.reshape(-1, 1))

# Split the data into training and testing sets
train_size = int(len(series_scaled) * 0.8)
test_size = len(series_scaled) - train_size
train, test = series_scaled[0:train_size], series_scaled[train_size:len(series_scaled)]

# Set parameters for TimeseriesGenerator
time_steps = 10
batch_size = 32

# Create TimeseriesGenerator for training and testing
train_generator = TimeseriesGenerator(train, train, length=time_steps, batch_size=batch_size)
test_generator = TimeseriesGenerator(test, test, length=time_steps, batch_size=batch_size)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(time_steps, 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(train_generator, epochs=50, validation_data=test_generator, verbose=1)

# Make predictions
y_pred_scaled = model.predict(test_generator)
y_pred = scaler.inverse_transform(y_pred_scaled)

# Inverse transform y_test for comparison
y_test_inverse = scaler.inverse_transform(test[time_steps:])

# Evaluate the model
mse = tf.keras.losses.MeanSquaredError()
print("Test MSE:", mse(y_test_inverse, y_pred).numpy())

# Plot the results
import matplotlib.pyplot as plt

plt.plot(y_test_inverse, color='blue', label='Actual')
plt.plot(y_pred, color='red', label='Predicted')
plt.title('Actual vs Predicted')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()


In [4]:
print?

Signature: print(*args, sep=' ', end='\n', file=None, flush=False)
Docstring:
Prints the values to a stream, or to sys.stdout by default.

sep
  string inserted between values, default a space.
end
  string appended after the last value, default a newline.
file
  a file-like object (stream); defaults to the current sys.stdout.
flush
  whether to forcibly flush the stream.
Type:      builtin_function_or_method

# multi variable 

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load your data
# Assuming data is a pandas DataFrame with multiple input columns and one output column
data = pd.read_csv('your_time_series_data.csv')

# Split data into input features (X) and output (y)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Feature Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Set parameters for TimeseriesGenerator
time_steps = 10
batch_size = 32

# Create TimeseriesGenerator for training and testing
train_generator = TimeseriesGenerator(X_train, y_train, length=time_steps, batch_size=batch_size)
test_generator = TimeseriesGenerator(X_test, y_test, length=time_steps, batch_size=batch_size)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(time_steps, X_train.shape[1])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(train_generator, epochs=50, validation_data=test_generator, verbose=1)

# Make predictions
y_pred_scaled = model.predict(test_generator)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Inverse transform y_test for comparison
y_test_inverse = scaler_y.inverse_transform(y_test[time_steps:])

# Evaluate the model
mse = tf.keras.losses.MeanSquaredError()
print("Test MSE:", mse(y_test_inverse, y_pred).numpy())

# Plot the results
import matplotlib.pyplot as plt

plt.plot(y_test_inverse, color='blue', label='Actual')
plt.plot(y_pred, color='red', label='Predicted')
plt.title('Actual vs Predicted')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()
